# FracFocus Database Cleanup

## Required Packages and Config

In [30]:
#Run the below line to install pyproj
#!conda install --name metis -c conda-forge pyproj

In [31]:
import pandas as pd
from pathlib import Path
import datetime
from datetime import timedelta
from pyproj import Proj, transform
import pickle
import re
import seaborn as sns

In [32]:
#Setup Configs
%matplotlib inline
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 25)
pd.set_option('display.precision', 3)

## Data Import

In [33]:
pathlist_chem = Path('data/fracfocuscsv/').glob('FracFocusRegistry*.csv')
pathlist_chem = sorted(pathlist_chem)

In [34]:
def csv_onboard(pathlist):
    dfs = []
    for path in pathlist:
        dfs.append(pd.read_csv(str(path),low_memory=False))
        print('Loaded ' + str(path))
    return pd.concat(dfs)      

In [35]:
chem_raw = csv_onboard(pathlist_chem)

Loaded data/fracfocuscsv/FracFocusRegistry_01.csv
Loaded data/fracfocuscsv/FracFocusRegistry_02.csv
Loaded data/fracfocuscsv/FracFocusRegistry_03.csv
Loaded data/fracfocuscsv/FracFocusRegistry_04.csv
Loaded data/fracfocuscsv/FracFocusRegistry_05.csv
Loaded data/fracfocuscsv/FracFocusRegistry_06.csv
Loaded data/fracfocuscsv/FracFocusRegistry_07.csv
Loaded data/fracfocuscsv/FracFocusRegistry_08.csv
Loaded data/fracfocuscsv/FracFocusRegistry_09.csv
Loaded data/fracfocuscsv/FracFocusRegistry_10.csv
Loaded data/fracfocuscsv/FracFocusRegistry_11.csv
Loaded data/fracfocuscsv/FracFocusRegistry_12.csv
Loaded data/fracfocuscsv/FracFocusRegistry_13.csv
Loaded data/fracfocuscsv/FracFocusRegistry_14.csv
Loaded data/fracfocuscsv/FracFocusRegistry_15.csv
Loaded data/fracfocuscsv/FracFocusRegistry_16.csv
Loaded data/fracfocuscsv/FracFocusRegistry_17.csv
Loaded data/fracfocuscsv/FracFocusRegistry_18.csv


In [36]:
chem_total = chem_raw.copy(deep=True)

## Simple Cleanup

In [37]:
def simple_clean(df):
    df.columns = [column.strip() for column in df.columns]
    
    df['JobStartDate'] = (pd.to_datetime(df['JobStartDate'],
                                                 format = '%m/%d/%Y %H:%M:%S %p',
                                                 errors = 'coerce'))

    df['JobEndDate'] = (pd.to_datetime(df['JobEndDate'],
                                                 format = '%m/%d/%Y %H:%M:%S %p',
                                                 errors = 'coerce'))
    return df

In [38]:
chem_total = simple_clean(chem_total)

In [39]:
chem_total.sample(5, random_state=5)

,UploadKey,JobStartDate,JobEndDate,APINumber,StateNumber,CountyNumber,OperatorName,WellName,Latitude,Longitude,Projection,TVD,TotalBaseWaterVolume,TotalBaseNonWaterVolume,StateName,CountyName,FFVersion,FederalWell,IndianWell,Source,DTMOD,PurposeKey,TradeName,Supplier,Purpose,SystemApproach,IsWater,PurposePercentHFJob,PurposeIngredientMSDS,IngredientKey,IngredientName,CASNumber,PercentHighAdditive,PercentHFJob,IngredientComment,IngredientMSDS,MassIngredient,ClaimantCompany,DisclosureKey
221514,e2110ca8-a61a-4c20-83a5-a4002f3acf62,2014-12-05 12:00:00,2014-12-08 12:00:00,42313311740000,42,313,MD America Energy LLC,Thomas 1H,30.867,-96.113,NAD27,8606.0,4.068e+06,0.0,Texas,Madison,2,False,False,NaN,NaN,1ce5261b-5d98-4f5c-b530-7f2845121c8a,CSA-10 (TCS-302),Economy Polymers,Clay Control,NaN,NaN,NaN,NaN,e00e781d-c7dd-4165-99af-740654336dc9,Choline Chloride,67-48-1,0.7,2.197e-03,NaN,True,75262.878,NaN,e2110ca8-a61a-4c20-83a5-a4002f3acf62
180196,ada80ff9-947c-49b1-a5b5-8f145cbee62a,2016-09-14 05:00:00,2016-09-20 05:00:00,35073253470000,35,73,"Staghorn Petroleum, LLC",Vader 18-05-23 1H,36.015,-97.703,NAD27,6791.0,1.378e+07,0.0,Oklahoma,Kingfisher,3,False,False,NaN,NaN,9a62b354-e464-41ba-bd33-d5a7431cf4e3,Other Ingredients,NaN,Other Chemicals,NaN,NaN,NaN,NaN,4f4b261a-a35f-434f-948a-fe6b98b6274e,Cinnamaldehyde,104-55-2,0.0,1.475e-04,NaN,False,0.000,NaN,ada80ff9-947c-49b1-a5b5-8f145cbee62a
239921,b22f6e26-138e-4d02-9af3-8f35a8d29806,2013-04-17 12:00:00,2013-04-18 12:00:00,15077219120000,15,77,SandRidge Energy,Shrack 3406 2-28H,37.065,-97.977,NAD27,4661.0,1.344e+06,NaN,Kansas,Harper,2,False,False,NaN,NaN,37ca2035-2d47-4a36-8d87-862f14478bc7,Hydrochloric Acid (15%),Archer,Acidizing,NaN,NaN,NaN,NaN,b2122e82-e275-42ab-af62-eb0c91ab9fef,thiourea-formaldehyde copolymer,68527-49-1,15.0,2.359e-04,None,True,28.052,NaN,b22f6e26-138e-4d02-9af3-8f35a8d29806
147258,c78ee80a-6274-4e90-b044-c9f404d92bed,2013-06-18 03:13:00,2013-06-19 12:00:00,42317377180000,42,317,Pioneer Natural Resources,MARTIN 4 #4,32.257,-102.024,NAD27,11580.0,1.461e+06,0.0,Texas,Martin,2,False,False,NaN,NaN,55cb01f2-5603-4001-a49e-725310b55fb7,NaN,NaN,Other Chemicals,NaN,NaN,NaN,NaN,0fbd399b-2552-4bbb-b95b-97b9400707cf,Sodium hydroxide - TS,1310-73-2,0.0,8.621e-05,NaN,False,0.000,NaN,c78ee80a-6274-4e90-b044-c9f404d92bed
142172,9d95ef19-5954-47a9-bcc3-0394687fe1cb,2014-03-08 12:00:00,2014-03-10 12:00:00,35003227680000,35,3,SandRidge Energy,Rose 2510 2-20H,36.623,-98.302,NAD27,6019.0,1.995e+06,0.0,Oklahoma,Alfalfa,2,False,False,NaN,NaN,2f7510f0-03b5-42f8-9081-c200217f718b,NaN,NaN,Other Chemicals,NaN,NaN,NaN,NaN,014fb288-da7e-4534-84e2-efab075e2fca,Water,7732-18-5,0.0,3.774e-02,NaN,False,0.000,NaN,9d95ef19-5954-47a9-bcc3-0394687fe1cb


## Chemical Analysis

In [40]:
chem_total.sample(5, random_state=5)

,UploadKey,JobStartDate,JobEndDate,APINumber,StateNumber,CountyNumber,OperatorName,WellName,Latitude,Longitude,Projection,TVD,TotalBaseWaterVolume,TotalBaseNonWaterVolume,StateName,CountyName,FFVersion,FederalWell,IndianWell,Source,DTMOD,PurposeKey,TradeName,Supplier,Purpose,SystemApproach,IsWater,PurposePercentHFJob,PurposeIngredientMSDS,IngredientKey,IngredientName,CASNumber,PercentHighAdditive,PercentHFJob,IngredientComment,IngredientMSDS,MassIngredient,ClaimantCompany,DisclosureKey
221514,e2110ca8-a61a-4c20-83a5-a4002f3acf62,2014-12-05 12:00:00,2014-12-08 12:00:00,42313311740000,42,313,MD America Energy LLC,Thomas 1H,30.867,-96.113,NAD27,8606.0,4.068e+06,0.0,Texas,Madison,2,False,False,NaN,NaN,1ce5261b-5d98-4f5c-b530-7f2845121c8a,CSA-10 (TCS-302),Economy Polymers,Clay Control,NaN,NaN,NaN,NaN,e00e781d-c7dd-4165-99af-740654336dc9,Choline Chloride,67-48-1,0.7,2.197e-03,NaN,True,75262.878,NaN,e2110ca8-a61a-4c20-83a5-a4002f3acf62
180196,ada80ff9-947c-49b1-a5b5-8f145cbee62a,2016-09-14 05:00:00,2016-09-20 05:00:00,35073253470000,35,73,"Staghorn Petroleum, LLC",Vader 18-05-23 1H,36.015,-97.703,NAD27,6791.0,1.378e+07,0.0,Oklahoma,Kingfisher,3,False,False,NaN,NaN,9a62b354-e464-41ba-bd33-d5a7431cf4e3,Other Ingredients,NaN,Other Chemicals,NaN,NaN,NaN,NaN,4f4b261a-a35f-434f-948a-fe6b98b6274e,Cinnamaldehyde,104-55-2,0.0,1.475e-04,NaN,False,0.000,NaN,ada80ff9-947c-49b1-a5b5-8f145cbee62a
239921,b22f6e26-138e-4d02-9af3-8f35a8d29806,2013-04-17 12:00:00,2013-04-18 12:00:00,15077219120000,15,77,SandRidge Energy,Shrack 3406 2-28H,37.065,-97.977,NAD27,4661.0,1.344e+06,NaN,Kansas,Harper,2,False,False,NaN,NaN,37ca2035-2d47-4a36-8d87-862f14478bc7,Hydrochloric Acid (15%),Archer,Acidizing,NaN,NaN,NaN,NaN,b2122e82-e275-42ab-af62-eb0c91ab9fef,thiourea-formaldehyde copolymer,68527-49-1,15.0,2.359e-04,None,True,28.052,NaN,b22f6e26-138e-4d02-9af3-8f35a8d29806
147258,c78ee80a-6274-4e90-b044-c9f404d92bed,2013-06-18 03:13:00,2013-06-19 12:00:00,42317377180000,42,317,Pioneer Natural Resources,MARTIN 4 #4,32.257,-102.024,NAD27,11580.0,1.461e+06,0.0,Texas,Martin,2,False,False,NaN,NaN,55cb01f2-5603-4001-a49e-725310b55fb7,NaN,NaN,Other Chemicals,NaN,NaN,NaN,NaN,0fbd399b-2552-4bbb-b95b-97b9400707cf,Sodium hydroxide - TS,1310-73-2,0.0,8.621e-05,NaN,False,0.000,NaN,c78ee80a-6274-4e90-b044-c9f404d92bed
142172,9d95ef19-5954-47a9-bcc3-0394687fe1cb,2014-03-08 12:00:00,2014-03-10 12:00:00,35003227680000,35,3,SandRidge Energy,Rose 2510 2-20H,36.623,-98.302,NAD27,6019.0,1.995e+06,0.0,Oklahoma,Alfalfa,2,False,False,NaN,NaN,2f7510f0-03b5-42f8-9081-c200217f718b,NaN,NaN,Other Chemicals,NaN,NaN,NaN,NaN,014fb288-da7e-4534-84e2-efab075e2fca,Water,7732-18-5,0.0,3.774e-02,NaN,False,0.000,NaN,9d95ef19-5954-47a9-bcc3-0394687fe1cb


In [41]:
def get_uniques(col):
    length = len(col.unique())
    name = col.name
    return str(name) + ': ' + str(length)

print(len(chem_total))

for column in chem_total.columns[20:]:
    print(get_uniques(chem_total[column]))

4439900
DTMOD: 1
PurposeKey: 1677443
TradeName: 23036
Supplier: 4304
Purpose: 7785
SystemApproach: 1
IsWater: 1
PurposePercentHFJob: 1
PurposeIngredientMSDS: 1
IngredientKey: 4352056
IngredientName: 18228
CASNumber: 2929
PercentHighAdditive: 60892
PercentHFJob: 2173748
IngredientComment: 16130
IngredientMSDS: 3
MassIngredient: 870098
ClaimantCompany: 517
DisclosureKey: 116517


In [42]:
unique_chems = (chem_total.groupby(['TradeName'])['PercentHFJob']
                .count()
                .reset_index()
                .sort_values(by=['PercentHFJob'],ascending=False))
unique_chems.head(10)

,TradeName,PercentHFJob
16008,Other Chemical(s),116077
22342,Water,67043
16013,Other Ingredients,44320
13479,Ingredients in Additive(s) (MSDS and non-MSDS),34075
11651,FRW-200,31820
11984,Fresh Water,29557
7642,CI-150,23321
17753,Proppant Transport,23306
14459,LoSurf-300D,21057
19436,Sand,15529


## Chemical Lookup

In [43]:
from collections import OrderedDict

prop_mesh_regex = OrderedDict([
    ('100-mesh',[r'(?i)100M',r'(?i)100 M']),
    ('12/20',[r'12/20',r'12-20']),
    ('16/30',[r'16/30',r'16-30']),
    ('20/40',[r'20/40',r'20-40']),
    ('30/50',[r'30/50',r'30-50']),
    ('40/70',[r'40/70',r'40-70']),
    ('40/140',[r'40/140',r'40-140']),
    ('other',[r'(?i)sand',r'(?i)crc',r'(?i)carbo',r'(?i)ceramic',r'(?i)rcs',r'(?i)nws',r'(?i)silica',r'(?i)prop'])
])

prop_type_regex = OrderedDict([
    ('rcs',[r'(?i)crc',r'(?i)rcs',r'(?i)resin']),
    ('ceramic',[r'(?i)carbo',r'(?i)ceramic']),
    ('sand',[item for sublist in prop_mesh_regex.values() for item in sublist])
])

purpose_regex = OrderedDict([
    ('friction',[r'(?i)friction']),
    ('corrosion',[r'(?i)corrosion']),
    ('biocide',[r'(?i)bio']),
    ('surfactant',[r'(?i)surfact']),
    ('acid',[r'(?i)acid']),
    ('clay',[r'(?i)clay']),
    ('iron',[r'(?i)iron']),
    ('gel',[r'(?i)gel',r'(?i)cross'])
])

In [44]:
def ranked_lookup(string,regex_dict):
    try:
        for key in regex_dict.keys():
            for regex in regex_dict[key]:
                if re.search(regex,string):
                    return key        
        return None
    except TypeError:
        return None

In [45]:
chem_total['sand_mesh'] = chem_total['TradeName'].apply(ranked_lookup,args=[prop_mesh_regex])
chem_total['sand_type'] = chem_total['TradeName'].apply(ranked_lookup,args=[prop_type_regex])
chem_total['purpose_type'] = chem_total['Purpose'].apply(ranked_lookup,args=[purpose_regex])

In [46]:
chem_total.sample(5)

,UploadKey,JobStartDate,JobEndDate,APINumber,StateNumber,CountyNumber,OperatorName,WellName,Latitude,Longitude,Projection,TVD,TotalBaseWaterVolume,TotalBaseNonWaterVolume,StateName,CountyName,FFVersion,FederalWell,IndianWell,Source,DTMOD,PurposeKey,TradeName,Supplier,Purpose,SystemApproach,IsWater,PurposePercentHFJob,PurposeIngredientMSDS,IngredientKey,IngredientName,CASNumber,PercentHighAdditive,PercentHFJob,IngredientComment,IngredientMSDS,MassIngredient,ClaimantCompany,DisclosureKey,sand_mesh,sand_type,purpose_type
66424,cf0b4336-714f-4df5-af49-98d0703b17ee,2013-05-22 12:00:00,2013-05-22 12:00:00,43047345610000,43,47,QEP Energy Company,WVX 10W-17-8-21,40.122,-109.575,NAD83,7932.0,2.808e+04,0.0,Utah,Uintah,2,False,False,NaN,NaN,ed7bc661-0c60-4c5f-afbd-df2c00a20e04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,b1c6ed73-67e7-496d-8e5b-572f66312446,Amine Salts,593-81-7,0.000e+00,3.151e-05,NaN,False,1.000,NaN,cf0b4336-714f-4df5-af49-98d0703b17ee,None,None,None
128616,0b4d8515-f915-4fb0-b780-c39725a44e50,2019-04-05 12:00:00,2019-04-18 12:00:00,42461410250000,42,461,XTO Energy/ExxonMobil,Aegis 19-18 1911AH,31.538,-101.790,NAD83,8980.0,1.924e+07,0.0,Texas,Upton,3,False,False,NaN,NaN,8568e5af-f936-499a-b4f9-5691880cc4bc,CI200-Corrosion Inhibitor,Economy Polymers,Corrosion Inhibitor,NaN,NaN,NaN,NaN,af6db35a-c2f2-4842-9904-35d01971ee0e,Methanol,67-56-1,6.000e+01,6.412e-04,None,True,1122.970,NaN,0b4d8515-f915-4fb0-b780-c39725a44e50,None,None,corrosion
161991,5f0e8e50-b730-4582-bc31-c3985350d444,2014-03-04 12:00:00,2014-03-15 12:00:00,42283343340000,42,283,EP Energy,Ritchie Farms 92H,28.444,-99.395,NAD27,7829.0,8.609e+06,0.0,Texas,La Salle,2,False,False,NaN,NaN,96e912dc-a61b-4afd-9cdd-edf1146b984e,NE-100,FTSI,Non-emulsifier,NaN,NaN,NaN,NaN,36bb7240-18c9-4297-a9c5-cf4f72622c04,Sulfuric Acid,7664-93-9,1.000e-02,4.370e-09,NaN,True,0.350,NaN,5f0e8e50-b730-4582-bc31-c3985350d444,None,None,None
226743,70084c30-561d-4ea1-ba45-d1dcaf44b4f3,2013-09-09 12:00:00,2013-10-08 12:00:00,42269328700000,42,269,Hunt Oil Company,Burnett S B Estate N 14R,33.486,-100.061,NAD27,5392.0,6.888e+04,0.0,Texas,King,2,False,False,NaN,NaN,a85e6e36-55c4-4ef2-9d7e-f69b83805619,NaN,NaN,Other Chemicals,NaN,NaN,NaN,NaN,d1d009b3-c31f-42cf-a8a6-09af94fa9439,Quaternary Ammonium Compound,Trade Secret,0.000e+00,1.461e-03,NaN,False,10.633,NaN,70084c30-561d-4ea1-ba45-d1dcaf44b4f3,None,None,None
191663,f7faebf7-18ea-49cf-aade-81ffafb29a15,2014-06-12 12:00:00,2014-06-14 12:00:00,42173364190000,42,173,Pioneer Natural Resources,CHEATHAM 19 #6,32.009,-101.709,NAD27,10530.0,1.245e+06,0.0,Texas,Glasscock,2,False,False,NaN,NaN,8817050b-a346-4de7-a3f8-aa9c99170a3a,"A286, B244B, B451, F112, H036, J218, J475, J58...",Schlumberger,"Acid Corrosion Inhibitor A286, Bactericide (My...",NaN,NaN,NaN,NaN,b4e71438-8bf0-4129-8ea0-846b7efc9181,Ethanol,64-17-5,1.000e-05,0.000e+00,NaN,False,0.000,NaN,f7faebf7-18ea-49cf-aade-81ffafb29a15,12/20,sand,friction


In [51]:
chem_total_bysand_mesh = chem_total.groupby(['APINumber','sand_mesh'])['PercentHFJob'].sum().reset_index()
chem_total_bysand_type = chem_total.groupby(['APINumber','sand_type'])['PercentHFJob'].sum().reset_index()
chem_total_bypurpose = chem_total.groupby(['APINumber','purpose_type'])['PercentHFJob'].sum().reset_index()


chem_total_bysand_mesh = chem_total_bysand_mesh.pivot(index='APINumber',
                                                          columns='sand_mesh',
                                                          values='PercentHFJob').reset_index()

chem_total_bysand_type = chem_total_bysand_type.pivot(index='APINumber',
                                                          columns='sand_type',
                                                          values='PercentHFJob').reset_index()

chem_total_bypurpose = chem_total_bypurpose.pivot(index='APINumber',
                                                          columns='purpose_type',
                                                          values='PercentHFJob').reset_index()

chem_total_bysand_mesh = chem_total_bysand_mesh.fillna(0)
chem_total_bysand_type = chem_total_bysand_type.fillna(0)
chem_total_bypurpose = chem_total_bypurpose.fillna(0)

In [52]:
chem_total_bypurpose.head(5)

purpose_type,APINumber,acid,biocide,clay,corrosion,friction,gel,iron,surfactant
0,4226932868,0.000,0.005,2.446,0.000,0.000,1.368e+00,0.000e+00,0.0
1,1007203590000,0.074,0.015,0.000,0.001,0.036,2.170e-03,3.470e-04,0.0
2,1007203600000,0.053,0.022,0.000,0.001,0.003,3.140e-04,5.000e-04,0.0
3,1007203610000,0.052,0.015,0.000,0.001,0.002,3.720e-04,2.970e-04,0.0
4,1007203620000,0.026,0.012,0.000,0.000,0.002,1.400e-04,2.250e-04,0.0


In [53]:
def to_pickle(var,name,dir_name='pickle'):
    with open('{}/{}.pickle'.format(dir_name,name), 'wb') as to_write:
        pickle.dump(var, to_write)

In [54]:
to_pickle(chem_total_bypurpose,'chem_purpose')
to_pickle(chem_total_bysand_mesh,'chem_sandmesh')
to_pickle(chem_total_bysand_type,'chem_sandtype')